In [1]:
from ipywidgets import HBox, Button, Layout, FloatProgress
from IPython.display import display
import hublib.ui as ui
import paramiko
import logging
import socket
import re
import random
import time, threading
import string

<IPython.core.display.Javascript object>

In [2]:
PARADIM_SERVER = 'gateway2.marcc.jhu.edu'    
PARADIM_USER = 'dmejia4@jhu.edu'
PARADIM_CODE = ''
PARADIM_PWD = 'n4n0P4r4d1m.'
PARADIM_PORT = 22
PARADIM_SESSION = None
thread_update = None
PARADIM_PBS_ID = 0
PARADIM_TIMER = 30

In [3]:
class ShellHandler:

    def __init__(self, val_code, val_user, val_pwd):        
        
        def inter_handler(title, instructions, prompt_list):
            resp = [] 
            for pr in prompt_list:
                if str(pr[0]).strip() == "Username:":
                    resp.append(val_user)
                elif str(pr[0]).strip() == "Password:":
                    resp.append(val_pwd)
                elif str(pr[0]).strip() == "Verification code:":
                    resp.append(val_code)
            return tuple(resp)

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((PARADIM_SERVER, PARADIM_PORT))
        ts = paramiko.Transport(self.sock)
        ts.connect()
        ts.auth_interactive_dumb(val_user, inter_handler)
        self.ssh = ts.open_session()
        self.ssh.invoke_shell()
        self.stdin = self.ssh.makefile('wb')
        self.stdout = self.ssh.makefile('r')  
        self.stderr = self.ssh.makefile_stderr('r')  
        self.initialized = False       

    def __del__(self):
        self.ssh.close()
        #self.sock.close()

    def execute(self, cmd):
        cmd = cmd.strip('\n')
        print (cmd)
        self.stdin.write(cmd + '\n')
        finish = 'end of stdOUT buffer. finished with exit status'
        echo_cmd = 'echo {} $?'.format(finish)
        self.stdin.write(echo_cmd + '\n')
        shin = self.stdin
        self.stdin.flush()

        shout = []
        sherr = []
        exit_status = 0
        for line in self.stdout:
            if str(line).startswith(cmd) or str(line).startswith(echo_cmd):
                # up for now filled with shell junk from stdin
                shout = []
            elif str(line).startswith(finish):
                # our finish command ends with the exit status
                exit_status = int(str(line).rsplit(maxsplit=1)[1])
                if exit_status:
                    # stderr is combined with stdout.
                    # thus, swap sherr with shout in a case of failure.
                    sherr = shout
                    shout = []
                break
            else:
                # get rid of 'coloring and formatting' special characters
                shout.append(re.compile(r'(\x9B|\x1B\[)[0-?]*[ -/]*[@-~]').sub('', line).
                             replace('\b', '').replace('\r', ''))

        # first and last lines of shout/sherr contain a prompt
        if shout and echo_cmd in shout[-1]:
            shout.pop()
        if shout and cmd in shout[0]:
            shout.pop(0)
        if sherr and echo_cmd in sherr[-1]:
            sherr.pop()
        if sherr and cmd in sherr[0]:
            sherr.pop(0)

        return shin, shout, sherr


class setInterval :
    def __init__(self,interval,action) :
        self.interval=interval
        self.action=action
        self.stopEvent=threading.Event()
        thread=threading.Thread(target=self.__setInterval)
        thread.start()

    def __setInterval(self) :
        nextTime=time.time()+self.interval
        while not self.stopEvent.wait(nextTime-time.time()) :
            nextTime+=self.interval
            self.action()

    def cancel(self) :
        self.stopEvent.set()



In [4]:
def GetInput( ):
    global in_input
    return in_input.value

def GetPbs( input_file ):
    global in_pbs
    return in_pbs.value + " < " + input_file

def GetPbsFile():
    global in_pbs_file
    return in_pbs_file.value
    
def GetInputFile():
    #in_input_file = "myinput.in"
    in_input_file = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
    return in_input_file + ".in"

def CreateConnection( event ):
    try :
        global in_pwd, in_code, in_user, PARADIM_SESSION, PARADIM_PBS_ID    
        val_pwd = in_pwd.value
        val_code = in_code.value
        val_user = in_user.value    
        if PARADIM_SESSION is None:
            PARADIM_SESSION = ShellHandler(val_code, val_user, val_pwd)
        print ("Connection created successfully")
    except:
        print ("ERROR: There has been an issue with the authentication")
    
def SendJob( event ):
    global PARADIM_SESSION
    if PARADIM_SESSION is not None:        
        input_file = GetInputFile();
        stdin, stdout, stderr = PARADIM_SESSION.execute("printf \"" + GetInput().replace('\n', '\\n') + "\" > " + input_file);
        stdin, stdout, stderr = PARADIM_SESSION.execute("printf \"" + GetPbs(input_file).replace('\n', '\\n') + "\" > " + GetPbsFile());
        stdin, stdout, stderr = PARADIM_SESSION.execute("qsub " + GetPbsFile());
        code = stdout[len(stdout)-1].strip('\n')
        if (code.isdigit()):
            print(code)
    else :
        print ("ERROR: A new connection is required")
    
def getStatus( code ):
    try :
        stdin, stdout, stderr = PARADIM_SESSION.execute("qstat -a " + str(code));
        line = stdout[len(stdout)-1].strip('\n')
        columns = line.split()
        if (len(columns) < 11):
            status = 'X'
        else :
            status = str(columns[9])
    except :
        status = 'X'
        line = "ERROR"
    return status, line

def AsyncCheck ( ):
    global inter, in_pbs_id,in_timer, in_auto_reload, last_status
    
    val_pbs_id = in_pbs_id.value
    val_auto_reload = in_auto_reload.value         
    status, response = getStatus( val_pbs_id )
    counter = 0
    
    if (status != last_status):
        print (response)
        last_status = status
        
    if status != 'E' and status != 'X' and status != 'C' and status != 'O' :
        if (val_auto_reload == False):
            t=threading.Timer(0,inter.cancel)
            t.start() 
            in_auto_reload.value = False
    else :
        t=threading.Timer(0,inter.cancel)
        t.start()
        in_auto_reload.value = False


def LoadingProgressUpdate():
    global progress_bar, in_timer, in_auto_reload
    total = 100
    progress_bar.value = 0
    i=0
    while True:
        if (i > total or in_auto_reload.value == False):
            i = 0
        if (int(in_timer.value) > 0):
            time.sleep(float(int(in_timer.value)/total))
            progress_bar.value = float(i+1)/total
            i = i + 1
        
def CheckStatus( event ):
    global in_timer, PARADIM_SESSION, inter, last_status
    if event['type'] == 'change' and event['name'] == 'value' and event['new']:   
        if PARADIM_SESSION is not None:        
            last_status = ''
            val_timer = int(in_timer.value)
            inter=setInterval(val_timer,AsyncCheck)
            AsyncCheck()
        else :
            print ("ERROR: A new connection is required")



In [5]:
in_pwd = ui.String('MARCC Password',PARADIM_PWD)
in_code = ui.String('GOOGLE AUTH Code',PARADIM_CODE)
in_user = ui.String('Marcc User name',PARADIM_USER)
in_button = Button(description='Connect')
in_button.on_click(CreateConnection)
tab = ui.Form([in_user, in_pwd, in_code, HBox([in_button])], name = 'MARCC Credentials')


in_input = ui.Text(
    name="inputdeck",
    value='''&control
 calculation = 'scf',
 prefix = 'silicon',
 pseudo_dir = './',
 outdir = './'
/
&system
 ibrav = 2,
 celldm(1) = 10.28,
 nat = 2,
 ntyp = 1,
 ecutwfc= 18.0,
/
&electrons
/
ATOMIC_SPECIES
 Si 28.086 Si.pz-vbc.UPF
ATOMIC_POSITIONS alat
 Si 0.00 0.00 0.00
 Si 0.25 0.25 0.25
K_POINTS automatic
 4 4 4 1 1 1'''
)
in_input.dd.layout = Layout(width='400px', height='400px')
tab_b = ui.Form([in_input], name = 'Simulation Inputs')


in_pbs_file = ui.String('PBS filename','mypbs.pbs')
in_pbs = ui.Text(
    name="pbs",
    value='''#!/bin/bash -l
#SBATCH
#SBATCH --job-name=job-1
#SBATCH --time=00:30:00
#SBATCH --partition=parallel
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=24
module load quantumespresso/6.2.1
mpirun -np 12 pw.x -npool 4'''
)
in_pbs.dd.layout = Layout(width='400px', height='400px')
in_job = Button(description='Send Job')
in_job.on_click(SendJob)
tab_c = ui.Form([in_pbs, in_pbs_file, HBox([in_job])], name = 'Jobs Configuration')


#in_pbs_id = ui.Integer('PBS ID', str(PARADIM_PBS_ID))
in_pbs_id = ui.String('PBS ID', str(PARADIM_PBS_ID))
#in_timer = ui.Integer('TIMER', str(PARADIM_TIMER))
in_timer = ui.String('TIMER', str(PARADIM_TIMER))
in_auto_reload = ui.Togglebuttons('AUTO RELOAD',value=False, options=[True, False])
in_auto_reload.dd.observe(CheckStatus)

progress_bar = FloatProgress(value=0.0, min=0.0, max=1.0)
if thread_update is None:
    thread_update = threading.Thread(target=LoadingProgressUpdate)
    thread_update.start()
tab_d = ui.Form([in_pbs_id, in_timer, in_auto_reload, progress_bar], name = 'Job Status')

tabs = ui.Tab([tab, tab_b, tab_c, tab_d])
display(tabs)
